<a href="https://colab.research.google.com/github/rajendradayma/SHL-Assessment-Recommender/blob/main/shl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 55.5 MB/s eta 0:00:00


In [6]:
!pip install streamlit # Corrected the typo in the package name

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 70.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.4 MB/s eta 0:00:00


In [7]:
import pandas as pd
import faiss
import streamlit as st
from sentence_transformers import SentenceTransformer

In [14]:
!pip install fastapi
!pip install pydantic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.6 MB/s eta 0:00:00


In [16]:
app

In [22]:
!pip install pyngrok # Install the pyngrok package

In [29]:
# SHL Assessment Recommendation Engine using RAG (Retrieval Augmented Generation)
# Streamlit frontend with SentenceTransformer + FAISS backend (Colab-compatible)

import streamlit as st
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import os
from pyngrok import ngrok

# === Load and Embed SHL Product Data ===
class SHLRecommender:
    def __init__(self, data_path):
        self.df = pd.read_csv(data_path)
        self.df.fillna("", inplace=True)
        self.texts = (
            self.df['Individual Test Solutions'] + ". " +
            self.df['Description'] + ". " +
            self.df['Job Levels'] + ". " +
            self.df['Test Type']
        ).tolist()

        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        self.embeddings = self.model.encode(self.texts, show_progress_bar=True)

        self.index = faiss.IndexFlatL2(self.embeddings.shape[1])
        self.index.add(self.embeddings)

    def query(self, user_query, top_k=5):
        query_embedding = self.model.encode([user_query])
        _, indices = self.index.search(query_embedding, top_k)
        results = self.df.iloc[indices[0]].to_dict(orient="records")
        return results

# === Streamlit App File Writing for Colab Compatibility ===
def write_app_file():
    with open("app.py", "w") as f:
        f.write('''
import streamlit as st
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss

class SHLRecommender:
    def __init__(self, data_path):
        self.df = pd.read_csv(data_path)
        self.df.fillna("", inplace=True)
        self.texts = (
            self.df['Individual Test Solutions'] + ". " +
            self.df['Description'] + ". " +
            self.df['Job Levels'] + ". " +
            self.df['Test Type']
        ).tolist()

        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        self.embeddings = self.model.encode(self.texts, show_progress_bar=True)

        self.index = faiss.IndexFlatL2(self.embeddings.shape[1])
        self.index.add(self.embeddings)

    def query(self, user_query, top_k=5):
        query_embedding = self.model.encode([user_query])
        _, indices = self.index.search(query_embedding, top_k)
        results = self.df.iloc[indices[0]].to_dict(orient="records")
        return results

recommender = SHLRecommender("/content/shl_catalog_detailed.csv")

st.set_page_config(page_title="SHL Assessment Recommender", layout="wide")
st.title("🔍 SHL Assessment Recommender")

user_query = st.text_input("Enter your job role, skills, or test topic:", "entry level python developer")

if st.button("Recommend Assessments") and user_query:
    with st.spinner("Searching for relevant assessments..."):
        recommendations = recommender.query(user_query)

    st.subheader("📋 Recommended Assessments")
    for rec in recommendations:
        st.markdown(f"**{rec['Individual Test Solutions']}**")
        st.markdown(f"- 🔗 [Link]({rec['URL']})")
        st.markdown(f"- 📄 Description: {rec['Description']}")
        st.markdown(f"- 💼 Job Levels: {rec['Job Levels']}")
        st.markdown(f"- 🧪 Test Type: {rec['Test Type']}")
        st.markdown("---")
''')

# === Setup ngrok tunnel and run Streamlit App ===
def launch_streamlit():
    write_app_file()
    from pyngrok import conf
    conf.get_default().auth_token = "2uoo6zGCOe7cYc2sYAtcrKSTUzO_3iT9iR82wD4N1TSY6GXE9"  # 👈 paste your token here

    ngrok.kill()
    public_url = ngrok.connect(8501)
    print("\n🌐 Streamlit app is live at:", public_url)
    os.system("streamlit run app.py &")


In [45]:
launch_streamlit()


🌐 Streamlit app is live at: NgrokTunnel: "https://41a4-34-168-14-53.ngrok-free.app" -> "http://localhost:8501"
